<a href="https://colab.research.google.com/github/rasecfaria/FinalProject/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importar bibliotecas
import pandas as pd
import numpy as np
import datetime

In [ ]:
#import dos filmes
movies = pd.read_csv('movies.csv')
#movies.head(10)

In [5]:
#import das avaliações
ratings = pd.read_csv('ratings.csv')
#ratings.head(10)

In [6]:
#FILTRAGEM DE DADOS
#remover o timesstamp dos ratings(provisorio, check notion)
ratings = ratings [['userId', 'movieId', 'rating']]
#ratings.head(10)

In [ ]:
#numero de avaliacoes de cada utilizador
ratings['userId'].value_counts()

In [ ]:
movies.info()

In [ ]:
ratings.info()

In [ ]:
#juncao dos dataframes
df = ratings.merge(movies, on='movieId')
df.head(10)

In [ ]:
#pivot quero que cada userID seja uma variave com o respetivo valor de nota para cada filme
movie_pivot = df.pivot_table(columns='userId', values='rating', index='title')
movie_pivot.head(10)

In [12]:
#preencher valores nulos com 0
movie_pivot.fillna(0, inplace=True)

In [13]:
#criar matrix sparsa
from scipy.sparse import csr_matrix
movie_sparse = csr_matrix(movie_pivot)

In [14]:
#importar o algoritmo KNN do Scikit
from sklearn.neighbors import NearestNeighbors

In [ ]:
#treinar o modelo
model = NearestNeighbors(algorithm='brute')
model.fit(movie_sparse)

Previsao de sugestao de filmes

In [18]:
#Toy Story
distances, suggestions = model.kneighbors(movie_pivot.filter(items = ['Toy Story (1995)'], axis=0).values.reshape(1, -1))

for i in range(len(suggestions)):
  print(movie_pivot.index[suggestions[i]])

Index(['Toy Story (1995)', 'Toy Story 2 (1999)', 'Mission: Impossible (1996)',
       'Independence Day (a.k.a. ID4) (1996)', 'Bug's Life, A (1998)'],
      dtype='object', name='title')


In [20]:
#velocidade furiosa
distances, suggestions = model.kneighbors(movie_pivot.filter(items = ['Fast and the Furious, The (2001)'], axis=0).values.reshape(1, -1))

for i in range(len(suggestions)):
  print(movie_pivot.index[suggestions[i]])

Index(['Fast and the Furious, The (2001)',
       '2 Fast 2 Furious (Fast and the Furious 2, The) (2003)',
       'Rundown, The (2003)', 'The Scorpion King (2002)', 'Mr. Deeds (2002)'],
      dtype='object', name='title')
